<a href="https://colab.research.google.com/github/TEAM-N0VA/strat-final/blob/yerin/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask pyngrok scikit-learn pandas numpy

from flask import Flask, request, jsonify
from pyngrok import ngrok
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
# ---------------------------------------------------------
# 토큰 입력
ngrok.set_auth_token("36qwBBGo052i1Ag8vijY4kogz50_3G7VhM6491v8NYpHNWU9x")
# ---------------------------------------------------------

app = Flask(__name__)
port = 5000
public_url = ngrok.connect(port).public_url
print(f"🚀 [서버 시작] {public_url}")

# 추천 데이터베이스
RECOMMENDATION_DB = {
    "OTT": ["디즈니+", "티빙", "라프텔"],
    "FOOD": ["요기요 패스", "쿠팡이츠 와우할인", "마켓컬리"],
    "SHOPPING": ["네이버 플러스 멤버십", "신세계 유니버스"],
    "MUSIC": ["유튜브 뮤직", "스포티파이"]
}

def analyze_comprehensive(df):

    # 1. 데이터 전처리 (집계)
    summary = df.groupby('serviceName').agg({
        'cost': 'sum',
        'timeMinutes': 'sum',
        'payment_count': 'sum',
        'category': 'first'
    }).reset_index()

    # 2. Feature Engineering (효율성 지수 생성)
    efficiency_scores = []
    for i, row in summary.iterrows():
        cat = row['category']
        cost = row['cost']
        count = row['payment_count']
        time = row['timeMinutes']

        # OTT는 시간, 커머스는 횟수 위주로 효율성 판단
        if cat in ['OTT', 'MUSIC']:
            score = cost / (time + 1)
        else:
            score = 10000 / (count + 0.1)
        efficiency_scores.append(score)

    summary['efficiency_index'] = efficiency_scores

    # 데이터가 너무 적으면 AI 분석 불가 -> 규칙 기반 처리로 넘김
    if len(summary) < 3:
        summary['cluster_label'] = 0
        summary['anomaly'] = 1 # 정상
    else:
        # 3. Scaling (정규화)
        scaler = StandardScaler()
        # 학습용 데이터: 비용, 시간, 횟수, 효율성지수
        features = summary[['cost', 'timeMinutes', 'payment_count', 'efficiency_index']]
        X_scaled = scaler.fit_transform(features)

        # 4. [복구] 최적의 K 찾기 (Silhouette Score)
        best_k = 2
        best_score = -1
        max_k = min(len(summary), 5)

        if max_k > 2:
            for k in range(2, max_k):
                km = KMeans(n_clusters=k, random_state=42)
                labels = km.fit_predict(X_scaled)
                score = silhouette_score(X_scaled, labels)
                if score > best_score:
                    best_score = score
                    best_k = k

        # 5. K-Means 클러스터링 실행
        kmeans = KMeans(n_clusters=best_k, random_state=42)
        summary['cluster_label'] = kmeans.fit_predict(X_scaled)

        # 6. [복구] Isolation Forest (이상치 탐지)
        # contamination=0.2 (하위 20%를 이상치로 간주)
        iso_forest = IsolationForest(contamination=0.2, random_state=42)
        summary['anomaly'] = iso_forest.fit_predict(X_scaled) # -1: 이상치, 1: 정상

    # 7. 리포트 생성 (AI 결과 + 규칙 해석)
    report_list = []
    category_spend = {}

    for i, row in summary.iterrows():
        svc = row['serviceName']
        cat = row['category']
        cost = row['cost']
        count = row['payment_count']
        is_anomaly = row.get('anomaly', 1) # 없으면 정상 처리

        # 페르소나용 지출 합계
        category_spend[cat] = category_spend.get(cat, 0) + cost

        status = "적정 사용"
        reason = "평범한 사용 패턴입니다."

        # [AI 판단] Isolation Forest가 이상치(-1)로 지목했나?
        if is_anomaly == -1:
            # 이상치 중에서도 '나쁜 쪽(고비용 저효율)'인지 확인
            if cost > 10000 and (count <= 1 and row['timeMinutes'] < 60):
                status = "🔴 낭비 경고 (AI 감지)"
                reason = "AI 분석 결과, 비용 대비 사용량이 현저히 낮습니다."
            elif count >= 5 or row['timeMinutes'] > 1000:
                status = "💎 압도적 효율 (AI 감지)"
                reason = "남들과 다른 압도적인 사용량을 보이고 있습니다."

        # [규칙 보완] AI가 놓친 경우를 위한 하드코딩 룰
        elif cat in ['FOOD', 'SHOPPING']:
            if count >= 5:
                status = "🟢 뽕뽑는 중"
                reason = f"이번 달 {count}회 이용! 배달비/배송비 본전 뽑았습니다."
            elif cost > 20000 and count <= 1:
                status = "🟡 주의 필요"
                reason = "결제 금액은 큰데 이용 횟수가 적습니다."
        else: # OTT/MUSIC
            if row['timeMinutes'] < 60 and cost > 5000:
                status = "🟡 주의 필요"
                reason = "사용 시간이 다소 부족합니다."

        report_list.append({
            "service": svc,
            "status": status,
            "reason": reason,
            "detail": f"{cost}원 | {count}회 | {row['timeMinutes']}분"
        })

    # 8. 페르소나 & 추천 (기존 로직 유지)
    if not category_spend:
        persona = "데이터 없음"
        recommendation = ""
    else:
        top_category = max(category_spend, key=category_spend.get)
        if top_category == "FOOD":
            persona = "🍗 배달비로 집 한 채 날린 미식가"
            rec_list = RECOMMENDATION_DB["FOOD"]
        elif top_category == "OTT":
            persona = "🎥 넷플릭스가 낳은 괴물 (OTT 마니아)"
            rec_list = RECOMMENDATION_DB["OTT"]
        elif top_category == "SHOPPING":
            persona = "📦 택배 기사님과 절친"
            rec_list = RECOMMENDATION_DB["SHOPPING"]
        else:
            persona = "🎧 24시간 BGM 리스너"
            rec_list = RECOMMENDATION_DB["MUSIC"]

        my_services = summary['serviceName'].tolist()
        final_recs = [r for r in rec_list if r not in my_services]
        recommendation = f"'{final_recs[0]}' 서비스는 어떠세요?" if final_recs else "이미 해당 분야 만렙입니다!"

    return persona, report_list, recommendation

@app.route("/analyze", methods=['POST'])
def analyze():
    try:
        json_data = request.json
        data_list = json_data.get('logs', [])
        df = pd.DataFrame(data_list)

        if not df.empty:
            persona, report, rec_msg = analyze_comprehensive(df)
        else:
            persona = "데이터 없음"
            report = []
            rec_msg = ""

        return jsonify({
            "user_persona": persona,
            "inefficiency_report": report,
            "recommendation": rec_msg
        })

    except Exception as e:
        print("Error:", e)
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(port=port)

🚀 [서버 시작] https://hymnodical-superadequately-esmeralda.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 13:58:44] "POST /analyze HTTP/1.1" 200 -
